IMPORTANT! Before beginning any lab assignment, be sure to **make your own copy** of the notebook and name it "lastname - Lab 8" or something similar.

# Lab 8: Error-Based Learning — Linear Regression and Regularization

In lecture we studied **error-based learning**: the idea that a parameterized model can be improved by repeatedly measuring how wrong its predictions are and nudging its weights in a direction that reduces that error. The simplest concrete instance of this framework is **linear regression**.

In this lab you will:

1. Fit simple and multiple linear regression models to a real astronomical dataset.
2. Carefully evaluate model quality using residual analysis and standard regression metrics.
3. Extend the ordinary regression loss function with penalty terms (**regularization**) to improve generalization to unseen data.
4. Treat the regularization strength as a **hyperparameter** and tune it systematically.


## Part A: Simple Linear Regression

### Background

Recall from lecture that the simplest error-based model is a **linear regression** that predicts a continuous target $\mathbb{M}_{\mathbf{w}}(\mathbf{d})$, which outside of your textbook  is more commonly denoted $\hat y$, from one or more descriptive features $\mathbf{d}$:

$$
\begin{align*}
\hat y = \mathbb{M}_{\mathbf{w}}(\mathbf{d}) &= \mathbf{w}[0] + \mathbf{w}[1]\mathbf{d}[1] + \mathbf{w}[2]\mathbf{d}[2] + \ldots + \mathbf{w}[n]\mathbf{d}[n]\\
&= \mathbf{w}[0] + \sum_{j=1}^n \mathbf{w}[j]\mathbf{d}[j]
\end{align*}
$$
If we define $\mathbf{d}[0] = 1$ as a dummy feature, we can re-write this more compactly as:
$$
\begin{align*}
\hat y &= \sum_{j=0}^n \mathbf{w}[j]\mathbf{d}[j]\\
&= \mathbf{w} \cdot \mathbf{d}
\end{align*}
$$

The weights $\mathbf{w}$ are chosen to minimize the **$L_2$ (sum-of-squared-errors) loss** over all the training examples:

$$L_2(\mathbf{w}) = \frac{1}{2} \sum_{i=1}^{N} \left(t_i - \hat{y}_i\right)^2$$

For linear regression this loss surface is convex, which guarantees that gradient descent (or the closed-form least-squares solution used by scikit-learn) will find the global minimum.


### The Business Knowledge

We will work with a real galaxy survey dataset drawn from the [Union2.1 Compilation](https://supernova.lbl.gov/union/).  Each row describes one galaxy which has had a Type Ia supernova observed in it.  Since all Type Ia supernovae are assumed to have the same intrinsic brightness, the "luminosity" distance is a good proxy for the actual distance.  The columns relevant to this lab are:

| Column | Description |
|--------|-------------|
| `mag`  | Apparent magnitude of supernova observed in the galaxy (i.e., how bright the supernova looks from Earth) |
| `z`    | Redshift — the fractional stretching of the galaxy's light due to the expansion of the universe         |
| `vel`     | Recessional velocity (in km/s) — how fast the galaxy is moving away from us (although cosmologists would say it is how fast the space between us and the galaxy is expanding) |
| `lum_dist` | Luminosity distance (in Mpc) — the distance to the galaxy estimated from the observed brightness of its supernova |

**We have limited this initial dataset to "nearby" galaxies within 1000 Mpc (about 3.26 billion light years).**  This dataset is a great testbed for regression because the underlying physics is well-understood: the recessional velocity of a galaxy is proportional to its distance from us, making it an excellent example of a linear relationship. This law is known as **Hubble's Law**, and it was one of the key pieces of evidence for the expansion of the universe.

### Step A.1: Load and Explore the Dataset

Now, let's load the dataset and explore it.

In [ ]:
import requests
from pathlib import Path
import polars as pl

# Set up Polars to show all columns when displaying DataFrames
pl.Config.set_tbl_formatting("ASCII_FULL_CONDENSED")
pl.Config.set_tbl_cols(-1)

# Set the URL and local filename
url = 'https://raw.githubusercontent.com/JuanCab/csis446_lab08/refs/heads/main/data/galaxies.csv'
local_filename = "galaxies.csv"

# Check if the file already exists to avoid re-downloading
if not Path(local_filename).is_file():
    print(f"Downloading dataset from {url}...")
    # Download the dataset using requests library
    r = requests.get(url)
    # Check if the request was successful, if not, raise an error
    r.raise_for_status()

    # Save the content to a local file
    with open(local_filename, "wb") as f:
        f.write(r.content)
        print(f"Dataset downloaded and saved as '{local_filename}'.")

# Load the dataset using Polars and display its shape and first few rows
galaxies_df = pl.read_csv(local_filename)
print(f'Dataset shape: {galaxies_df.shape}')
galaxies_df.head(10)

In [ ]:
# Summary statistics using Polars
galaxies_df.describe()

**Question 1**: Examine the summary statistics above.

- What is the range of recessional velocities (`vel`) in the dataset?
- The column `z` is the **redshift** of each galaxy.  Based on the summary statistics, what is the maximum redshift in the dataset?
- Which feature do you expect to have the **strongest linear relationship** with `lum_dist`, and why? (**Hint**: Think of the business knowledge presented earlier.)

```
ANSWER HERE
```

### Step A.2: Visualize Relationships

Before fitting any model it is essential to look at the data. The cell below creates a **pair plot** showing scatter plots for every pair of columns.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Remember we converta Polars dataframe to Pandas format so we can use
# Seaborn's pairplot
_ = sns.pairplot(galaxies_df.to_pandas())
plt.suptitle('Pair Plot: Galaxy Survey Dataset', y=1.02)
plt.tight_layout()
plt.show()

# Correlation matrix
corr_matrix = galaxies_df.to_pandas().corr()
plt.figure(figsize=(8, 6))
print("Correlation Matrix:")
print(corr_matrix)


**Question 2a**: Examine the pair plot carefully.

- Which feature(s) appears to have the **strongest and most clearly linear** relationship with `lum_dist`?  Does this match your prediction from Question 1?


```
ANSWER HERE
```

### Additional Business Information

Redshift (`z`) can be measured by getting a galaxy's spectrum with a telescope.  The redshift (`z`) itself is the shift in spectral lines caused by Doppler shifting of the galaxy's light.  You may have heard Doppler shifting in the context of sound waves — for example, the change in pitch of a passing ambulance siren.  The same principle applies to light waves, but instead of changing pitch we see a change in color.  The `vel` is defined as speed of light times `z`:
$$
vel = c \cdot z
$$
So the more its light is shifted toward the red end of the spectrum (the bigger the `z`) the faster the galaxy is moving away from us.

**Question 2b:**  You should notice that `vel` and `z` are perfectly correlated with each other.  Why should we have expected this?  Why would `vel` vs. `lum_dist` and `z` vs. `lum_dist` look essentially the same in the pair plot?  Why might including both as predictors in a regression model cause problems?


```
ANSWER HERE
```

### Still More Business Information

The only quantities that were actually directly measured for these supernova were `mag` and `z`.  The `vel` column is computed from `z` using Doppler shift formula.  The `lum_dist` column is computed from `mag` assuming all galaxies have the same intrinsic brightness (which is not actually true, but it's a common simplifying assumption in astronomy).

**Question 2c**: Does the relationship between `mag` and `lum_dist` look linear? Is it correlated? 

```
ANSWER HERE
```

**Question 2d**: Why should we probably drop both `mag` and `z` as predictors in a regression model to predict `lum_dist`?

```
ANSWER HERE
```

### Step A.3: Train / Test Split

We start with the single most promising feature for a linear relationship from Question 2: `lum_dist` should depend on `vel` (an infact, based on the pair plots and correlation matrix results, we know it does). This relationship is the heart of Hubble's Law.  We will fit a simple linear regression model to predict `vel` from `lum_dist` alone.

Before fitting any model we split the data into a **training set** (used to learn the weights) and a **test set** (held out to evaluate how well the model generalises to unseen data). We will use a 70 / 30 split throughout the lab.

In [ ]:
from sklearn.model_selection import train_test_split
import numpy as np

# scikit-learn requires pandas / numpy, so convert once here
galaxies_pd = galaxies_df.to_pandas()

# Pull only the `vel` feature for now
X = galaxies_pd[['lum_dist']].to_numpy()
y = galaxies_pd['vel'].to_numpy()

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.30,
    random_state=42
)

print(f'Training set size : {X_train.shape[0]:,} rows')
print(f'Test     set size : {X_test.shape[0]:,} rows')

**Question 3**: Why do we evaluate model performance on the **test set** rather than the training set?  What could go wrong if we used training-set performance as our only measure of model quality?

```
ANSWER HERE
```

### Step A.4: Fit a Simple Linear Regression Model

Let's try to predict `vel` from `lum_dist` using a simple linear regression model. We will then evaluate the model's performance on the test set using standard regression metrics and residual analysis.

In [ ]:
from sklearn.linear_model import LinearRegression

# Simple linear regression: rel_dist ~ vel
simple_lr = LinearRegression()
simple_lr.fit(X_train, y_train)

print(f'Intercept  w[0] : {simple_lr.intercept_:.6f}')
print(f'Slope      w[1] : {simple_lr.coef_[0]:.4e}')

In [ ]:
# Plot training data and the fitted line
# [:, 0] slices all rows but only the first column (vel) since X_train 
# is technically a 2D array with one column.
vel_range = np.linspace(X_train[:, 0].min(), X_train[:, 0].max(), 300).reshape(-1, 1)
dist_pred_line = simple_lr.predict(vel_range)

fig, ax = plt.subplots(figsize=(8, 5))
ax.scatter(X_train[:, 0], y_train, alpha=0.4, s=20, label='Training data')
labelstr = f'vel = {simple_lr.intercept_:.2f} + {simple_lr.coef_[0]:.2e} * lum_dist'
ax.plot(vel_range, dist_pred_line, color='crimson', linewidth=2, label=labelstr)
ax.set_xlabel('Luminosity Distance (Mpc)')
ax.set_ylabel('Recessional Velocity (km/s)')
ax.set_title('Velocity vs. Distance with Linear Fit')
ax.legend()
plt.tight_layout()
plt.show()

**Question 4**: Interpret the fitted model.

- What does the slope $w_1$ tell you in physical terms?  (Think about what it means for the relationship between a galaxy's speed and its distance — this is Hubble's Law in action.)
- The intercept $w_0$ gives the predicted `lum_dist` when `vel = 0`.  Does this make physical sense?  What would it mean for a galaxy to have zero recessional velocity?
- Using just the fitted weights, predict the recessional velocity of a galaxy at a luminosity distance of 100 Mpc.



```
ANSWER HERE
```


### Step A.5: Evaluate with Metrics and Residual Analysis

A regression model's performance is captured by several complementary metrics:

| Metric | Formula | Intuition |
|--------|---------|----------|
| **MSE** | $\frac{1}{N}\sum(t_i - \hat{y}_i)^2$ | Average squared error; penalizes large errors heavily |
| **RMSE** | $\sqrt{\text{MSE}}$ | Same units as the target; easier to interpret |
| **MAE** | $\frac{1}{N}\sum\|t_i - \hat{y}_i\|$ | Average absolute error; more robust to outliers |
| **$R^2$** | $1 - \frac{\text{SS}_{\text{res}}}{\text{SS}_{\text{tot}}}$ | Proportion of variance explained by the model (0 = none, 1 = all). |

A bit of explanation here, variance is the total squared deviation of the target values from their mean and essentially measures the spread of the data. If the model explains all the variance, then the residual sum of squares ($SS_{res}$) will be zero and $R^2$ will be 1. If the model just predicts the mean of the target values for every example, then $SS_{res}$ will be equal to $SS_tot$ and $R^2$ will be 0 (that is the model doesn't explain any variance).

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

def regression_metrics(y_true, y_pred, label='Model'):
    """Print a tidy summary of regression metrics and return them as a dict."""
    mse  = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    mae  = mean_absolute_error(y_true, y_pred)
    r2   = r2_score(y_true, y_pred)
    print(f'--- {label} ---')
    print(f'  RMSE : {rmse:.6f}')
    print(f'  MAE  : {mae:.6f}')
    print(f'  R^2  : {r2:.6f}')
    return {'rmse': rmse, 'mae': mae, 'r2': r2}

# Baseline: always predict the training-set mean
y_baseline    = np.full_like(y_test, fill_value=y_train.mean(), dtype=float)
baseline_rmse = np.sqrt(mean_squared_error(y_test, y_baseline))
baseline_mae  = mean_absolute_error(y_test, y_baseline)
baseline_r2   = r2_score(y_test, y_baseline)
regression_metrics(y_test, y_baseline, label='Baseline (predict mean)')

print()

# Simple linear regression on test set
y_pred_simple = simple_lr.predict(X_test[:, 0].reshape(-1, 1))
simple_metrics = regression_metrics(y_test, y_pred_simple, label='Simple LR (vel only)')

Another common way to evaluate regression models is to look at the **residuals** — the differences between the true target values and the model's predictions.  A good regression model should have residuals that are randomly scattered around zero, with no clear patterns.

In [ ]:
# Residual analysis
residuals_simple = y_test - y_pred_simple

fig, axes = plt.subplots(1, 2, figsize=(12, 4))

# Residuals vs. fitted values
axes[0].scatter(y_pred_simple, residuals_simple, alpha=0.4, s=20)
axes[0].axhline(0, color='crimson', linewidth=1.5, linestyle='--')
axes[0].set_xlabel('Fitted values (predicted vel)')
axes[0].set_ylabel('Residual (actual - predicted)')
axes[0].set_title('Residuals vs. Fitted Values')

# Histogram of residuals
axes[1].hist(residuals_simple, bins=40, edgecolor='white')
axes[1].axvline(0, color='crimson', linewidth=1.5, linestyle='--')
axes[1].set_xlabel('Residual')
axes[1].set_ylabel('Count')
axes[1].set_title('Distribution of Residuals')

plt.suptitle('Residual Analysis - Simple Linear Regression (lum_dist only)', fontsize=13)
plt.tight_layout()
plt.show()

**Question 6**: Analyze the residual plots and metrics.

- The simple model uses only `vel` and yet achieves a high $R^2$.  Does this surprise you?  What does it tell us about how informative a single well-chosen feature can be?
- What would a **perfect** residuals-vs-fitted plot look like?  Describe any systematic patterns you notice in this one — is the spread of residuals consistent at all predicted distances, or does it fan out?
- Is the residual distribution approximately normal and centered on zero?  What would a long tail in one direction imply about the model's errors? How is this model doing in that regard?



```
ANSWER HERE
```

## Part B: Fitting Non-Linear Relationships

### More Business Knowledge

Astronomers in the late-20th Century expected that since the universe has mass, over time, its own gravity would cause the expansion to slow down.  This would mean that the relationship between recessional velocity and distance would not be perfectly linear, but would instead show a slight curve as we look at more distant galaxies.  

**Why would Hubble's Law not be perfectly linear?**

Accept for a moment that the slope of a `vel` vs. `lum_dist` relationship tells us the rate at which the universe is expanding (since it says how fast space is expanding per unit distance).  If the expansion is slowing down, we would expect as we look at more distant galaxies (which are also older galaxies) the slope of `vel` vs. `lum_dist` would **increase** in the past, since the universe was expanding faster in the past when those galaxies emitted the light we are now seeing.  This would create a slight curve in the `vel` vs. `lum_dist` relationship, with more distant galaxies showing higher velocities than what a simple linear model would predict.

Without going into the physics, the model predicted a specific form of this curve:
$$
\begin{align*}
vel &= H_0 \cdot lum\_dist \cdot \left(1 - \frac{(1-q_0)}{2} \frac{H_0}{c}lum\_dist\right)\\
    & = H_0 \cdot lum\_dist + \frac{H_0^2 (q_0-1)}{2c}lum\_dist^2\\
\end{align*}
$$
where $H_0$ is the Hubble constant (the slope of the linear relationship) and $q_0$ is the "deceleration parameter" that quantifies how much the expansion is slowing down.  If $q_0$ is zero the universe's expansion is not slowing down at all and the relationship is perfectly linear.  The key point $H_0$ was our previous slope (about 61.8 km/s/Mpc) and $q_0$ was expected to be positive (indicating deceleration) so we could re-write this expression above as
$$
vel = w_0 + w_1 \cdot lum\_dist + w_2 \cdot lum\_dist^2
$$
where $w_0$ is the intercept (expected to be zero), $w_1$ is the slope of the linear term (the slope of our original linear model), and $w_2$ is the coefficient of the quadratic term.  If the universe's expansion was slowing down as expected, then
the second weight $w_2$ should have been positive, creating a curve that bends upwards as we look at more distant galaxies.

We can create a hypothetical model below just to show what we would predict if the universe's expansion was slowing down as expected.  The curve bends upwards, showing that more distant galaxies were in a universe that was expanding faster than what a simple linear model would predict.

In [ ]:
# Predicted curve (no actual numbers here, just the shape of the curve)
w0 = simple_lr.intercept_
w1 = simple_lr.coef_[0]
w2 = 0.1  # Example quadratic coefficient

x = np.linspace(0, 10000, 250)
y = w0 + w1 * x + w2 * x**2

plt.plot(x, y, color='crimson', linewidth=2)
plt.xlabel('Luminosity Distance (Mpc)')
plt.ylabel('Recessional Velocity (km/s)')
plt.title('Hypothetical Non-Linear Relationship')

**Question 7**: Given our data before only extended to distances of 1000 Mpc, why would we not have been able to detect this curvature in the `vel` vs. `lum_dist` relationship?  What does this tell us about the importance of having data that covers a wide range of feature values when trying to fit non-linear relationships?

```
ANSWER HERE
```

### Step B.1: Get a More Complete Dataset and Explore

The dataset of galaxies within 1000 Mpc was a subset of a larger dataset that includes galaxies considerably farther out.  Let's load the full dataset and see if we can detect the curvature predicted by the model above.

In [ ]:
# Load the complete dataset of galaxy data

# Set the URL and local filename
url = 'https://raw.githubusercontent.com/JuanCab/csis446_lab08/refs/heads/main/data/galaxies_fullSCPunion2.1.csv'
local_filename = "galaxies_fullset.csv"

# Check if the file already exists to avoid re-downloading
if not Path(local_filename).is_file():
    print(f"Downloading dataset from {url}...")
    # Download the dataset using requests library
    r = requests.get(url)
    # Check if the request was successful, if not, raise an error
    r.raise_for_status()

    # Save the content to a local file
    with open(local_filename, "wb") as f:
        f.write(r.content)
        print(f"Dataset downloaded and saved as '{local_filename}'.")

# Load the dataset using Polars and display its shape and first few rows
galaxies_full_df = pl.read_csv(local_filename)
print(f'Dataset shape: {galaxies_full_df.shape}')
galaxies_full_df.head(10)

**Question 8**: Examine the statistics for this full dataset, and compare them to the statistics for the original dataset.  Do you notice any differences in the ranges of `vel` and `lum_dist`?  How might these differences affect our ability to fit a non-linear model?

In [ ]:
# TODO: Summary statistics of galaxies_df using Polars

In [ ]:
# TODO: Summary statistics galaxies_full_df using Polars

```
ANSWER HERE
```

Let's plot up the `vel` vs. `lum_dist` relationship for this full dataset and see if we can visually detect any curvature.

In [ ]:
# TODO: Plot up 'lum_dist' vs. 'vel' for the full dataset

**Question 9**: Notice that for the full dataset the curvature is much more apparent, with the points starting to show a slight curve as we look at more distant galaxies.  Does this match the predictions of the model above?

```
ANSWER HERE
```

### Step B.2: Add the new non-linear features

To fit a non-linear relationship, we can simply add new features that are non-linear transformations of the original features.  Recall we expected a relationship in the form:
$$
\text{vel} = w[0] + w[1] \times \text{lum\_dist} + w[2] \times \text{lum\_dist}^2
$$
To fit this quadratic relationship we can add a new feature that is the square of `lum_dist`.  This allows us to capture the curvature in the data while still using a linear regression model.

In [ ]:
# Add lum_dist^2 as a new feature to the full dataset
galaxies_full_df = galaxies_full_df.with_columns(
    (galaxies_full_df['lum_dist'] ** 2).alias('lum_dist_squared')
)
galaxies_full_df.head(10)

### Step B.3: Fit the Non-Linear Model (and a Linear Baseline)

Now let's use linear regression to fit both a linear model and a non-linear model (by including the new `lum_dist_squared` feature), and evaluate their performance on the test set.

In [ ]:
# Create the feature matrix X and target vector y for the full dataset
X_full = galaxies_full_df[['lum_dist', 'lum_dist_squared']].to_numpy()
y_full = galaxies_full_df['vel'].to_numpy()

# TODO: Construct a test/train split for the full dataset with 30%
# held for training and a random state of 42 for reproducibility.
Xf_train, Xf_test, yf_train, yf_test = ...

# Print the sizes of the training and test sets for the full dataset
print(f'Training set size : {Xf_train.shape[0]:,} rows')
print(f'Test     set size : {Xf_test.shape[0]:,} rows')

In [ ]:
# Fit a linear model to see how well it does (using only the first
# feature) as a baseline for comparison to the non-linear model
linear_lr = LinearRegression()
linear_lr.fit(Xf_train[:, 0].reshape(-1, 1), yf_train)
print("Linear model results:")
print(f'Linear model intercept: {linear_lr.intercept_:.6f}')
print(f'Linear model coefficient: {linear_lr.coef_[0]:.4e}')

# Fit the non-linear model using linear regression on the full dataset 
# with the new feature
nonlinear_lr = LinearRegression()
nonlinear_lr.fit(Xf_train, yf_train)
print("\nNon-linear model results:")
print(f'Intercept  w[0] : {nonlinear_lr.intercept_:.6f}')
print(f'Coefficient w[1] : {nonlinear_lr.coef_[0]:.4e} (lum_dist)')
print(f'Coefficient w[2] : {nonlinear_lr.coef_[1]:.4e} (lum_dist_squared)')

And let's plot the data with the fitted non-linear regression curve to visually assess how well it captures the curvature in the data.

In [ ]:
# Plot the training data with the fitted non-linear regression curve
lum_dist_range = np.linspace(Xf_train[:, 0].min(), Xf_train[:, 0].max(), 300)
lum_dist_squared_range = lum_dist_range ** 2
X_range = np.column_stack((lum_dist_range, lum_dist_squared_range))
vel_pred_nonlinear = nonlinear_lr.predict(X_range)
vel_pred_linear = linear_lr.predict(lum_dist_range.reshape(-1, 1))

fig, ax = plt.subplots(figsize=(8, 5))
ax.scatter(Xf_train[:, 0], yf_train, alpha=0.5, s=20, label='Training data')
labelstr_linear = (f'vel = {linear_lr.intercept_:.2f} + {linear_lr.coef_[0]:.2e} * lum_dist')
ax.plot(lum_dist_range, vel_pred_linear, color='blue', linewidth=2,
        alpha=0.5, label=labelstr_linear)
labelstr = (f'vel = {nonlinear_lr.intercept_:.2f} + {nonlinear_lr.coef_[0]:.2e} * lum_dist + '
            f'{nonlinear_lr.coef_[1]:.2e} * lum_dist^2')
ax.plot(lum_dist_range, vel_pred_nonlinear, color='crimson', linewidth=2, label=labelstr)
ax.set_xlabel('Luminosity Distance (Mpc)')
ax.set_ylabel('Recessional Velocity (km/s)')
ax.set_title('Velocity vs. Distance with Non-Linear Fit')
ax.legend()
plt.tight_layout()
plt.show()

**Question 10**: Notice the $w[2]$ coefficient for the non-linear model is much smaller than it's $w[1]$ coefficient. Explain why we should have expected this based on our previous simple linear model. **HINT**: The $w[2]$ coefficient captures the size of the quadratic term.

```
ANSWER HERE
```

#### Step B.4: Evaluate the Non-Linear Model

Let's evaluate the performance of the non-linear model versus the linear baseline model using the same metrics and residual analysis as before.

In [ ]:
# Recall we have defined a regression_metrics function to compute and
# print the RMSE, MAE, and R^2 metrics for regression models.
#    regression_metrics(y_true, y_pred, label='Model')

# TODO: Evaluate the linear model on the test set
y_pred_linear_test = linear_lr.predict(Xf_test[:, 0].reshape(-1, 1))
linear_metrics = ...

# TODO: Evaluate the non-linear model on the test set
y_pred_nonlinear_test = nonlinear_lr.predict(Xf_test)
nonlinear_metrics = ...

In [ ]:
# Residual analysis
residuals_linear = yf_test - y_pred_linear_test
residuals_nonlinear = yf_test - y_pred_nonlinear_test

fig, axes = plt.subplots(2, 2, figsize=(12, 8))

# Residuals vs. fitted values for the linear model
axes[0, 0].scatter(y_pred_linear_test, residuals_linear, alpha=0.4, s=20)
axes[0, 0].axhline(0, color='crimson', linewidth=1.5, linestyle='--')
axes[0, 0].set_xlabel('Fitted values (predicted vel)')
axes[0, 0].set_ylabel('Residual (actual - predicted)')
axes[0, 0].set_title('Residuals vs. Fitted Values (Linear Model)')

# Histogram of residuals for the linear model
axes[0, 1].hist(residuals_linear, bins=40, edgecolor='white')
axes[0, 1].axvline(0, color='crimson', linewidth=1.5, linestyle='--')
axes[0, 1].set_xlabel('Residual')
axes[0, 1].set_ylabel('Count')
axes[0, 1].set_title('Distribution of Residuals')

# Residuals vs. fitted values for the nonlinear model
axes[1, 0].scatter(y_pred_nonlinear_test, residuals_nonlinear, alpha=0.4, s=20)
axes[1, 0].axhline(0, color='crimson', linewidth=1.5, linestyle='--')
axes[1, 0].set_xlabel('Fitted values (predicted vel)')
axes[1, 0].set_ylabel('Residual (actual - predicted)')
axes[1, 0].set_title('Residuals vs. Fitted Values (Nonlinear Model)')

# Histogram of residuals for the nonlinear model
axes[1, 1].hist(residuals_nonlinear, bins=40, edgecolor='white')
axes[1, 1].axvline(0, color='crimson', linewidth=1.5, linestyle='--')
axes[1, 1].set_xlabel('Residual')
axes[1, 1].set_ylabel('Count')
axes[1, 1].set_title('Distribution of Residuals (Nonlinear Model)')

plt.suptitle('Residual Analysis', fontsize=13)
plt.tight_layout()
plt.show()

**Question 11**: Can you definitively say that the non-linear model is better than the linear model based on the metrics and residual plots?  Why or why not?  What would you want to see in the metrics and residuals to be more confident that the non-linear model is truly capturing a real relationship in the data rather than just overfitting?

```
ANSWER HERE
```

## Part C: Regularization to Improve Generalization

Linear Regression performed extremely well for this dataset because the relationship is SO STRONG and there are so many more data points than parameters to fit that it can find the best solution easily.  However, in many real-world datasets, the relationships are more subtle and there may be many more features than data points.  In these cases, linear regression can easily overfit the training data, leading to poor generalization to unseen data.

### Background

Recall that a **loss function** quantifies the difference between a model's predictions and the observed values. For regression, the most common loss function is the **$L_2$ loss** (also known as the sum of squared errors), which measures the average squared difference between the predicted values and the true target values:

$$L_2(\mathbf{w}) = \frac{1}{2} \sum_{i=1}^{N} \left(t_i - \hat{y}_i\right)^2$$

When fitting the model, the algorithm has to solve for the weights that minimize this loss function. Using this approach, linear regression can fail to  fit well in some situations:

1. Imagine we have a large outlier in the data for `lum_dist`, by squareing the distance in the quadratic term, this outlier would have a disproportionate influence on the fitted model, potentially leading to overfitting (where we fit the noise in the data rather than the underlying relationship).
2. Imagine we instead have a dataset with a large number of features and the features show **colinearity** (where features are highly correlated). In this case, the model can become unstable and produce large weights for some features, which can also lead to overfitting and poor generalization to unseen data.

Both of these cases, if we could adjust weights downward to reduce the influence of outliers or to reduce the influence of certain colinear features, we could improve generalization.  This is the motivation for regularization.

### What is Regularization?

 **Regularization** modifies the loss function by adding a penalty term that discourages large weights:

| Method | Modified loss | Key property |
|--------|-------------|--------|
| **Ridge** ($\ell_2$) | $L_2 + \alpha \sum_j w_j^2$ | Shrinks all weights smoothly toward zero; never exactly zero |
| **LASSO** ($\ell_1$) | $L_2 + \alpha \sum_j \|w_j\|$ | Drives some weights to **exactly** zero - which leads to implicit feature selection |
| **Elastic Net** | $L_2 + \alpha \left[ \lambda \sum_j \|w_j\| + (1-\lambda)\sum_j w_j^2 \right]$ | Blend of Ridge and LASSO |

where

- $\alpha$ controls **how strongly** the penalty is applied where $\alpha = 0$ implements an ordinary $L_2$ loss function, and larger $\alpha$ forces weights closer to zero.  
-  $\lambda$ (written `l1_ratio` in scikit-learn) in Elastic Net blends between the two penalty types — `l1_ratio=0` is pure Ridge, `l1_ratio=1` is pure LASSO.

### Motivation

Let's now work with a dataset that does have some issues, the California rentals dataset we used a few weeks ago in class.  This dataset has more features and the relationships are more subtle, so we can see the benefits of regularization in action.

We will use a real dataset of apartment rental listings (`rent18.csv`). Each row is one listing, and the columns of interest are:

| Column | Description |
|--------|-------------|
| `price` | Monthly rent in USD (target variable $y$) |
| `sqft`  | Apartment size in square feet |
| `beds`  | Number of bedrooms |
| `baths` | Number of bathrooms |

Let's load the dataset and take a look at the first few rows, as well as some summary statistics.

In [ ]:
# Set the URL and local filename
url = 'https://raw.githubusercontent.com/JuanCab/csis446_lab08/refs/heads/main/data/rent18.csv'
local_filename = "rent18.csv"

# Check if the file already exists to avoid re-downloading
if not Path(local_filename).is_file():
    print(f"Downloading dataset from {url}...")
    # Download the dataset using requests library
    r = requests.get(url)
    # Check if the request was successful, if not, raise an error
    r.raise_for_status()

    # Save the content to a local file
    with open(local_filename, "wb") as f:
        f.write(r.content)
        print(f"Dataset downloaded and saved as '{local_filename}'.")

# Load with Polars, keep only the four columns of interest, drop all
# instances with missing values (recognize "NA" as nulls), and display
# the first few rows to verify successful loading
#
# NOTE: We are just dropping all rows with nulls.  We are not aiming for
# an unbiased dataset here, just something we can play with.
rent_raw = pl.read_csv(local_filename,
                       infer_schema_length=None,
                       null_values=["NA"])
rent_df = rent_raw.select(['sqft', 'beds', 'baths', 'price']).drop_nulls()
rent_df

In [ ]:
# Since we are using seaborn, we need to convert the polars dataframe
# to pandas.  This is so we can use the very useful pairplot function to
# visualize the relationships between all pairs of variables.
_ = sns.pairplot(rent_df.to_pandas())
plt.suptitle('Pair Plot: Rental Price Dataset', y=1.02)
plt.tight_layout()
plt.show()

# Also compute correlation matrix to quantify the strength of linear 
# relationships
corr_matrix = rent_df.to_pandas().corr()
print("Correlation Matrix:")
print(corr_matrix)


### Step C.1: Standardize the Features

Because the penalty is applied directly to weight magnitudes, all features must be **standardized** (zero mean, unit variance) before fitting a regularized model. Otherwise, features measured in large units (like `vel`, which reaches tens of thousands) would be penalized far more harshly than features with small numerical scales.

In [ ]:
# Split the rental dataset into training and testing data
feature_names = rent_df.drop(['price']).columns
X_rent = rent_df[feature_names].to_numpy()
y_rent = rent_df['price'].to_numpy()

Xrent_train, Xrent_test, yrent_train, yrent_test = train_test_split(
    X_rent, y_rent,
    test_size=0.30,
    random_state=42
)

print(f'Training set size : {Xrent_train.shape[0]:,} rows')
print(f'Test     set size : {Xrent_test.shape[0]:,} rows')

In [ ]:
from sklearn.preprocessing import StandardScaler

# Standardize using training-set statistics only by fitting the scaler
# on the training set and transforing the training set (done with a single
# fit_transform()) and then applying the same transformation to the
# test set (using transform() only, without refitting the scaler).
scaler     = StandardScaler()
X_train_sc = scaler.fit_transform(Xrent_train) # fit AND transform on training set
X_test_sc  = scaler.transform(Xrent_test)      # transform only on test set

print('Feature means before scaling (training):', Xrent_train.mean(axis=0).round(4))
print('Feature stds  before scaling (training):', Xrent_train.std(axis=0).round(4))
print()
print('Feature means after  scaling (training):', X_train_sc.mean(axis=0).round(8))
print('Feature stds  after  scaling (training):', X_train_sc.std(axis=0).round(8))

**Question 12**: Why do we call `fit_transform` on the **training set** but only `transform` on the **test set**?  What would go wrong if we standardized using the statistics of the combined training + test data?


```
ANSWER HERE
```

Let's first perform an ordinary (albeit multivariate) linear regression on this dataset without any regularization, and evaluate its performance on the test set.

In [ ]:
# First, fit the unregularized model on the SCALED features so we have
# a fair baseline for comparing weights
unregularized_lr = LinearRegression()
unregularized_lr.fit(X_train_sc, yrent_train)

print('Unregularized model on scaled features:')
for feature, coef in zip(feature_names, unregularized_lr.coef_):
    print(f'Coefficient for {feature:6s} : {coef:.4e}')

# Print RMSE
yrent_pred = unregularized_lr.predict(X_test_sc)
mse  = mean_squared_error(yrent_test, yrent_pred)
rmse = np.sqrt(mse)
print(f"RMSE: ${rmse:,.2f}")

### Step C.2: Ridge Regression ($\ell_2$ Regularization)

Ridge regression shrinks all weights smoothly toward zero as $\alpha$ increases, but never sets them exactly to zero.

In [ ]:
from sklearn.linear_model import Ridge, Lasso, ElasticNet
# Sweep alpha values for Ridge and record test RMSE and weights
alpha_values = np.logspace(-2, 4, 200)   # from 0.01 to 10000

# List and dict to save data in
ridge_rmse   = []
ridge_weights = {name: [] for name in feature_names}

for a in alpha_values:
    mdl = Ridge(alpha=a)
    mdl.fit(X_train_sc, yrent_train)
    preds = mdl.predict(X_test_sc)
    ridge_rmse.append(np.sqrt(mean_squared_error(yrent_test, preds)))
    for name, coef in zip(feature_names, mdl.coef_):
        ridge_weights[name].append(coef)

# Identify best fit
best_alpha_ridge = alpha_values[np.argmin(ridge_rmse)]
best_rmse_ridge  = min(ridge_rmse)
print(f'Best Ridge alpha : {best_alpha_ridge:.4f}')
print(f'Best Ridge RMSE  : ${best_rmse_ridge:,.2f}')

# Print feature weights for best fit
print('Ridge regression coefficients for best alpha:')
best_idx = np.argmin(ridge_rmse)
for name in feature_names:
    print(f'{name:15s}: {ridge_weights[name][best_idx]:8.3f}')

As you can see, this did outperform the previous model, but just barely!

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(13, 4))

# RMSE vs alpha
axes[0].semilogx(alpha_values, ridge_rmse, color='steelblue')
axes[0].axvline(best_alpha_ridge, color='crimson', linestyle='--',
                label=f'Best α = {best_alpha_ridge:.2f}')
axes[0].set_xlabel('α (regularization strength)')
axes[0].set_ylabel('Test RMSE ($)')
axes[0].set_title('Ridge: Test RMSE vs α')
axes[0].legend()

# Weights vs alpha
colors = ['tab:blue', 'tab:orange', 'tab:green']
for name, color in zip(feature_names, colors):
    axes[1].semilogx(alpha_values, ridge_weights[name], label=name, color=color)
axes[1].axvline(best_alpha_ridge, color='crimson', linestyle='--',
                label=f'Best α = {best_alpha_ridge:.2f}')
axes[1].axhline(0, color='black', linewidth=0.5)
axes[1].set_xlabel('α (regularization strength)')
axes[1].set_ylabel('Weight value')
axes[1].set_title('Ridge: Weight Paths vs α')
axes[1].legend()

plt.suptitle('Ridge Regression ($\\ell_2$ Regularization)', fontsize=13)
plt.tight_layout()
plt.show()

**Question 13**: Look at the two Ridge plots above.

- Describe what happens to the **weights** as $\alpha$ increases from left to right.  Do they all shrink at the same rate?
- Describe the shape of the **RMSE curve**: why does RMSE rise again at very large values of $\alpha$?
- What is the best $\alpha$ found and the corresponding RMSE?  How does this compare to the unregularized model?

```
ANSWER HERE
```

### Step C.3: LASSO Regression ($\ell_1$ Regularization)

LASSO uses a $\ell_1$ penalty that geometrically encourages exact zeros — it performs implicit **feature selection** by driving unimportant weights all the way to zero.

In [ ]:
# Set up list/dict to store fitting information
lasso_rmse   = []
lasso_weights = {name: [] for name in feature_names}

# Sweep alpha values for Lasso and record test RMSE and weights
for a in alpha_values:
    mdl = Lasso(alpha=a, max_iter=10000)
    mdl.fit(X_train_sc, yrent_train)
    preds = mdl.predict(X_test_sc)
    lasso_rmse.append(np.sqrt(mean_squared_error(yrent_test, preds)))
    for name, coef in zip(feature_names, mdl.coef_):
        lasso_weights[name].append(coef)

# Output best alpha and RMSE for Lasso
best_alpha_lasso = alpha_values[np.argmin(lasso_rmse)]
best_rmse_lasso  = min(lasso_rmse)
print(f'Best LASSO alpha : {best_alpha_lasso:.4f}')
print(f'Best LASSO RMSE  : ${best_rmse_lasso:,.2f}')

# Output the weights for this model
print('LASSO regression coefficients for best alpha:')
best_idx = np.argmin(lasso_rmse)
for name in feature_names:
    print(f'{name:15s}: {lasso_weights[name][best_idx]:8.3f}')

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(13, 4))

axes[0].semilogx(alpha_values, lasso_rmse, color='darkorange')
axes[0].axvline(best_alpha_lasso, color='crimson', linestyle='--',
                label=f'Best α = {best_alpha_lasso:.2f}')
axes[0].set_xlabel('α (regularization strength)')
axes[0].set_ylabel('Test RMSE ($)')
axes[0].set_title('LASSO: Test RMSE vs α')
axes[0].legend()

for name, color in zip(feature_names, colors):
    axes[1].semilogx(alpha_values, lasso_weights[name], label=name, color=color)
axes[1].axvline(best_alpha_lasso, color='crimson', linestyle='--',
                label=f'Best α = {best_alpha_lasso:.2f}')
axes[1].axhline(0, color='black', linewidth=0.5)
axes[1].set_xlabel('α (regularization strength)')
axes[1].set_ylabel('Weight value')
axes[1].set_title('LASSO: Weight Paths vs α')
axes[1].legend()

plt.suptitle('LASSO Regression ($\\ell_1$ Regularization)', fontsize=13)
plt.tight_layout()
plt.show()

**Question 14**: Compare the LASSO weight paths to the Ridge weight paths.

- A key property of LASSO is that it drives some weights to **exactly zero**.  Which feature's weight reaches zero first as $\alpha$ increases?  What does this imply about that feature's usefulness for predicting rent?
- Why is the ability to produce exactly-zero weights useful compared to Ridge's smooth shrinkage?  In what kind of problem would you prefer LASSO over Ridge?

```
ANSWER HERE
```

### Step C.4: Elastic Net Regression

Elastic Net combines both penalties.  Its loss is:

$$L_{\text{ENet}} = L_{2} + \alpha \left[ \lambda \sum_j |w_j| + (1-\lambda)\sum_j w_j^2 \right]$$

In scikit-learn, $\alpha$ is called `alpha` and $\lambda$ is called `l1_ratio`.  When `l1_ratio=0` you get pure Ridge; when `l1_ratio=1` you get pure LASSO.  We will explore a small grid of `l1_ratio` values.

In [ ]:
# Lambdas to test for Elastic Net
l1_ratios = [0.1, 0.5, 0.9]

# Dictionary for storing results for each l1_ratio.
# Each entry will be a tuple of (rmse list, weight dict)
enet_results = {}

# Fit a grid of Elastic Net models for each l1_ratio and record RMSE
# and weights
for l1r in l1_ratios:
    rmse_list = []
    w_dict = {name: [] for name in feature_names}
    for a in alpha_values:
        mdl = ElasticNet(alpha=a, l1_ratio=l1r, max_iter=10000)
        mdl.fit(X_train_sc, yrent_train)
        preds = mdl.predict(X_test_sc)
        rmse_list.append(np.sqrt(mean_squared_error(yrent_test, preds)))
        for name, coef in zip(feature_names, mdl.coef_):
            w_dict[name].append(coef)
    enet_results[l1r] = (rmse_list, w_dict)

# Print best RMSE for each l1_ratio
for l1r, (rmse_list, _) in enet_results.items():
    best_a = alpha_values[np.argmin(rmse_list)]
    best_r = min(rmse_list)
    print(f'Elastic Net l1_ratio={l1r:.1f} -> best α={best_a:.4f}, RMSE=${best_r:,.2f}')

**Question 15**: Looking at the Elastic Net results, did it perform better than Ridge or LASSO.  Which model would you ultimately choose in this case?

```
ANSWER HERE
```

## What you need to submit

Once you have completed this lab, you need to submit your work. You should submit the `.ipynb` notebook file. To do this, go to the File menu and select **Download > Download .ipynb** (this is the native Jupyter notebook format).  Submit that file to the Lab 8 dropbox on D2L.